# Instructional Notebook for SHRED + Biomechanics

The following lines can be uncommented if running this notebook in Google Colab. Uncomment by highlighting lines and pressing Ctrl+/

In [3278]:
#from google.colab import drive
#drive.mount('/content/drive')
#!pip install mat73
#!git clone https://github.com/Jan-Williams/pyshred
#%cd /content/pyshred

These lines import standard packages for managing data.

In [7]:
import os
import numpy as np
import altair as alt
import pandas as pd
from processdata import TimeSeriesDataset
import models
import torch
from sklearn.preprocessing import MinMaxScaler
from scipy.io import loadmat
import mat73
import functions as ft

# *** Update ***
Load a subject's data and manipulate the dataframe to be "tidy" = one row per time step and one column per signal. Depending on the dataset, loading and managing data will look different.

## Obtain subject's experimental data

Items to adjust before running a trial:

*   save_df - True (save) or False (don't save)
*   subject - '##'
*   activity code - AC## (this may need a different identifier depending on the dataset, just need a way to distinguish running speeds)

In [89]:
# Change subject number
subj = '01'
save_df = True   # True: save SHRED output dataframes only, False: don't save
trial_length = 5  # in minutes, accepts integers 1-6
frequency = 128 # in Hz, accepts integers up to 128

# adjust file path for saving if parameters are modified from 6min or 128Hz
if trial_length == 6:
    save_tag = str(frequency)+'Hz'
elif frequency == 128:
    save_tag = str(trial_length)+'min'
 

## Import Matlab file structure with subject's experimental data

Access directories where data is stored and will be saved. Manually set up folders before running the code block to ensure known file paths.

In [17]:
trial_path= 'Ingraham_IMU_reconstruction_nowrist_running'
cwd = os.getcwd()
main_path = os.path.dirname(cwd) + '/Datasets' 

# Alternatively, use the below lines if using Colab
# main_path = '/content/drive/MyDrive/Colab_Notebooks/Datasets'
# main_path = cwd+'/Datasets'

dataset_path = main_path+'/Data' # sets path to dataset / raw data
dataframe_path = main_path+'/Dataframes'  # file path for saved dataframe results of test data
figure_path = main_path+'/Figures'
model_path = main_path+'/Models' # optionally, save the models that are trained
print(dataset_path)
print(dataframe_path)


/Users/davidgreen/SHRED/Datasets/Ingraham/Data
/Users/davidgreen/SHRED/Datasets/Dataframes


Note on changing file directory and needing to save the parent directory:

https://stackoverflow.com/questions/14462833/how-can-i-go-back-to-the-previous-working-directory-after-changing-it

In [41]:
# load .mat file into pandas dataframe
load_mat = mat73.loadmat(dataset_path+'/Subject'+subj+'.mat')['Subject'+subj]
df = pd.DataFrame.from_dict(load_mat)

The example dataset contains several activities, two of which are 'Walking' and 'Running'. Access each independently.

In [85]:
#df_tmp = pd.DataFrame(data=df['Walking']['APDM_Accel']['Data'],
#                      columns = df['Walking']['APDM_Accel']['Labels'])

df_tmp = pd.DataFrame(data=df['Running']['APDM_Accel']['Data'],
                      columns = df['Running']['APDM_Accel']['Labels'])

pd.set_option('display.max_columns', None)

df_tmp.tail(5) # check that correct data was selected

Time (s) Activity Code                Waist                      \
                                  Acceleration (m/s^2)                       
                                                     x         y         z   
231348  1807.343463          12.0            -9.353293  1.222394  1.625345   
231349  1807.351275          12.0            -9.335909  1.123459  1.540691   
231350  1807.359088          12.0            -9.171153  1.077339  1.206564   
231351  1807.366900          12.0            -9.017789  1.329124  1.277886   
231352  1807.374712          12.0            -8.865735  1.374722  1.454045   

                                                                         \
       Angular Velocity (rad/s)                     Magnetic Field (uT)   
                              x         y         z                   x   
231348                -0.142169  0.034614 -0.177125           50.843437   
231349                -0.148348  0.037640 -0.184895           50.714705   
231350                -0.128062  0.063893 -0.182946           50.640699   
231351                -0.073453  0.047206 -0.184080           50.525998   
231352                 0.042053  0.055382 -0.189070           50.544571   

                                            Chest                      \
                             Acceleration (m/s^2)                       
                y          z                    x         y         z   
231348  16.872015  10.648010            -9.088658  0.150357  1.608253   
231349  17.094845  10.726108            -8.898073  0.133779  1.682710   
231350  17.056767  10.871868            -8.852520  0.135237  1.756955   
231351  17.037792  10.712959            -8.971994 -0.000197  1.824701   
231352  17.160503  10.715923            -8.925372 -0.223933  1.943991   

                                                                         \
       Angular Velocity (rad/s)                     Magnetic Field (uT)   
                              x         y         z                   x   
231348                 0.440451  0.272910  0.389406           57.418567   
231349                 0.425002  0.231876  0.373033           57.305999   
231350                 0.397494  0.157865  0.343987           57.243980   
231351                 0.355106  0.073336  0.299004           57.061209   
231352                 0.329611 -0.029720  0.263512           56.825190   

                                      Left Ankle                      \
                            Acceleration (m/s^2)                       
               y          z                    x         y         z   
231348  1.518668  14.197069           -10.376046 -0.368902 -2.140184   
231349  1.607555  14.145851           -10.403057 -0.227233 -2.169853   
231350  1.473774  14.260109           -10.562581 -0.652918 -2.124238   
231351  1.496668  14.406160           -10.797644 -0.895918 -1.633860   
231352  1.494085  14.216662           -10.658891 -0.008163 -1.353547   

                                                                         \
       Angular Velocity (rad/s)                     Magnetic Field (uT)   
                              x         y         z                   x   
231348                 0.639215 -0.079757 -0.241094           32.060207   
231349                 0.665515 -0.081127 -0.240671           31.953617   
231350                 0.692417 -0.089017 -0.254645           31.504868   
231351                 0.726948 -0.105717 -0.295743           31.344335   
231352                 0.788216 -0.114308 -0.310793           31.309810   

                                     Right Ankle                      \
                            Acceleration (m/s^2)                       
                y         z                    x         y         z   
231348  21.965298 -1.408134           -13.431479  1.032572 -0.510666   
231349  22.193238 -1.619954           -14.667708  1.500816 -1.404407   
231350  22.171354 -1.640882           -15.673375  2.739823 -2

In [30]:
# remove units and simplify column titles
columns_str = ["_".join(df_tmp.columns[i]).replace(" ", "") for i in np.linspace(0,df_tmp.shape[1]-1,df_tmp.shape[1]).astype(int)]
df_tmp.columns = columns_str

l_replace = [df_tmp.columns[i].replace('(m/s^2)', '') for i in np.linspace(0,df_tmp.shape[1]-1,df_tmp.shape[1]).astype(int)]
df_tmp.columns = l_replace

l_replace = [df_tmp.columns[i].replace('(rad/s)', '') for i in np.linspace(0,df_tmp.shape[1]-1,df_tmp.shape[1]).astype(int)]
df_tmp.columns = l_replace

l_replace = [df_tmp.columns[i].replace('(uT)', '') for i in np.linspace(0,df_tmp.shape[1]-1,df_tmp.shape[1]).astype(int)]
df_tmp.columns = l_replace
df_tmp.head()

,Time(s)__,ActivityCode__,Waist_Acceleration_x,Waist_Acceleration_y,Waist_Acceleration_z,Waist_AngularVelocity_x,Waist_AngularVelocity_y,Waist_AngularVelocity_z,Waist_MagneticField_x,Waist_MagneticField_y,Waist_MagneticField_z,Chest_Acceleration_x,Chest_Acceleration_y,Chest_Acceleration_z,Chest_AngularVelocity_x,Chest_AngularVelocity_y,Chest_AngularVelocity_z,Chest_MagneticField_x,Chest_MagneticField_y,Chest_MagneticField_z,LeftAnkle_Acceleration_x,LeftAnkle_Acceleration_y,LeftAnkle_Acceleration_z,LeftAnkle_AngularVelocity_x,LeftAnkle_AngularVelocity_y,LeftAnkle_AngularVelocity_z,LeftAnkle_MagneticField_x,LeftAnkle_MagneticField_y,LeftAnkle_MagneticField_z,RightAnkle_Acceleration_x,RightAnkle_Acceleration_y,RightAnkle_Acceleration_z,RightAnkle_AngularVelocity_x,RightAnkle_AngularVelocity_y,RightAnkle_AngularVelocity_z,RightAnkle_MagneticField_x,RightAnkle_MagneticField_y,RightAnkle_MagneticField_z,LeftFoot_Acceleration_x,LeftFoot_Acceleration_y,LeftFoot_Acceleration_z,LeftFoot_AngularVelocity_x,LeftFoot_AngularVelocity_y,LeftFoot_AngularVelocity_z,LeftFoot_MagneticField_x,LeftFoot_MagneticField_y,LeftFoot_MagneticField_z,RightFoot_Acceleration_x,RightFoot_Acceleration_y,RightFoot_Acceleration_z,RightFoot_AngularVelocity_x,RightFoot_AngularVelocity_y,RightFoot_AngularVelocity_z,RightFoot_MagneticField_x,RightFoot_MagneticField_y,RightFoot_MagneticField_z
0,0.007812,22.0,-9.846344,0.171822,1.227401,0.091794,-0.027342,-0.014275,46.815190,16.071594,13.902695,-9.764181,1.311812,0.985503,0.070709,0.084543,0.012162,55.879245,-1.209833,11.792871,-9.741885,-0.468814,-0.882530,0.118064,0.017438,0.021202,31.514212,18.934442,-8.637949,-9.838516,-0.574557,-1.155564,-0.006665,0.099050,0.009824,24.682745,-21.892916,18.683383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.015624,22.0,-9.775612,0.050246,1.209962,0.094758,-0.032213,-0.006748,46.996207,16.216266,13.923646,-9.757018,1.316749,0.992521,0.067192,0.088808,0.013694,55.828066,-1.158919,11.847663,-9.729948,-0.520553,-0.887037,0.113823,0.015991,0.017926,31.112110,18.884349,-8.560825,-9.851844,-0.576608,-1.153336,-0.003499,0.100780,0.011414,24.586635,-21.917585,18.649075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.023437,22.0,-9.835819,0.125845,1.167702,0.097836,-0.038619,-0.010032,46.728334,16.212284,13.792188,-9.754793,1.332469,0.997156,0.065436,0.091731,0.010487,55.859044,-1.419488,11.842671,-9.728399,-0.466783,-0.891656,0.116528,0.015885,0.019570,31.061167,19.012600,-8.561372,-9.856227,-0.571967,-1.148940,-0.003592,0.095519,0.009830,24.610019,-21.911564,18.789673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.031249,22.0,-9.810564,0.107856,1.203309,0.086923,-0.035422,-0.006915,46.646590,16.073072,13.775756,-9.757444,1.331871,0.987872,0.081388,0.086023,0.012167,55.757635,-1.280665,11.819464,-9.723669,-0.453419,-0.884954,0.113933,0.020142,0.017943,31.082365,19.015587,-8.637877,-9.849360,-0.574299,-1.151208,-0.002224,0.080491,0.011503,24.576289,-21.911981,18.789370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.039061,22.0,-9.800482,0.050737,1.143245,0.091569,-0.041706,-0.007019,46.688921,16.195991,13.996201,-9.746306,1.337988,0.989957,0.081584,0.084634,0.013785,55.632360,-1.164697,11.939039,-9.716266,-0.484920,-0.887274,0.108888,0.018438,0.019454,31.019347,18.862127,-8.912165,-9.845130,-0.569882,-1.144508,-0.008403,0.088848,0.009861,24.728641,-21.912624,18.651281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Obtain data with desired activity code

In [31]:
# subject running codes [12, 13, 14] = 1.8, 2,2, 2.7 m/s
AC_path = 'AC13'
df_2=df_tmp.loc[df_tmp['ActivityCode__']==13].dropna(axis=1,how='all')
df_2

,Time(s)__,ActivityCode__,Waist_Acceleration_x,Waist_Acceleration_y,Waist_Acceleration_z,Waist_AngularVelocity_x,Waist_AngularVelocity_y,Waist_AngularVelocity_z,Waist_MagneticField_x,Waist_MagneticField_y,Waist_MagneticField_z,Chest_Acceleration_x,Chest_Acceleration_y,Chest_Acceleration_z,Chest_AngularVelocity_x,Chest_AngularVelocity_y,Chest_AngularVelocity_z,Chest_MagneticField_x,Chest_MagneticField_y,Chest_MagneticField_z,LeftAnkle_Acceleration_x,LeftAnkle_Acceleration_y,LeftAnkle_Acceleration_z,LeftAnkle_AngularVelocity_x,LeftAnkle_AngularVelocity_y,LeftAnkle_AngularVelocity_z,LeftAnkle_MagneticField_x,LeftAnkle_MagneticField_y,LeftAnkle_MagneticField_z,RightAnkle_Acceleration_x,RightAnkle_Acceleration_y,RightAnkle_Acceleration_z,RightAnkle_AngularVelocity_x,RightAnkle_AngularVelocity_y,RightAnkle_AngularVelocity_z,RightAnkle_MagneticField_x,RightAnkle_MagneticField_y,RightAnkle_MagneticField_z
92163,720.003125,13.0,-8.672181,2.078084,-0.908607,-0.234768,0.155397,0.107042,47.523168,15.217146,15.122535,-11.415506,4.083131,1.037156,-0.452412,0.410344,-0.570006,55.346533,-0.108457,13.616935,-10.947737,0.333462,0.744122,1.257120,0.200747,-1.120145,37.006054,15.041447,-2.217998,-10.804503,-3.569351,-4.017712,-0.847475,-0.365022,2.486091,8.717718,-38.305682,15.262245
92164,720.010937,13.0,-9.108029,2.133242,-0.683718,-0.291168,0.176433,0.131376,47.627563,15.227421,14.967977,-12.320686,2.414399,1.109483,-0.467487,0.141405,-0.649037,55.503687,-0.067671,13.709523,-10.701714,3.421871,0.891543,0.890291,0.223097,-1.106815,36.727032,15.390673,-2.434100,-7.967302,-1.283468,-2.874473,-0.756204,-0.495349,2.290166,8.355993,-38.810741,14.838898
92165,720.018749,13.0,-9.760841,1.969770,-1.003231,-0.300523,0.209405,0.133393,47.607582,15.240576,15.058001,-12.700545,0.705280,1.090840,-0.351472,-0.068570,-0.671838,55.581175,-0.000368,13.870110,-10.412279,2.644220,0.084441,0.508470,0.246948,-1.127388,36.365086,15.613750,-2.552741,-7.477775,2.873775,-2.737740,-0.209013,-0.358341,2.504645,8.122922,-39.061494,14.714946
92166,720.026561,13.0,-10.180698,2.006988,-0.765093,-0.267819,0.216838,0.142912,47.622553,15.171323,15.425158,-12.632084,-0.747579,0.973582,-0.131381,-0.158783,-0.614230,55.638467,0.200683,13.634893,-10.038323,-0.037055,-0.571743,0.244053,0.289898,-1.152336,36.580776,15.962767,-2.393125,-7.235668,9.822751,-4.729407,-0.101764,0.009942,2.497063,7.794629,-39.752146,14.373450
92167,720.034374,13.0,-10.439398,1.925194,-0.551398,-0.181868,0.198508,0.126400,47.600302,14.948637,15.472936,-12.245499,-1.869060,0.846985,0.182224,-0.167330,-0.521108,55.880013,0.265159,13.324110,-9.650171,-1.058125,-0.913648,0.201461,0.321850,-1.099975,36.474189,16.427965,-2.443824,-3.995199,11.325384,-6.154482,-0.328885,0.157186,1.833636,7.191067,-40.271313,14.172405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138240,1079.965626,13.0,-9.172976,-0.071055,0.946888,0.886438,-0.051979,-0.049732,48.758513,21.075537,16.244491,-10.439755,-2.226243,0.697521,1.214866,0.828242,0.970492,54.600461,-6.479572,13.343407,-13.206731,-2.503830,-3.527720,-1.010973,0.489969,8.172326,21.508113,35.098387,-5.653059,-20.794520,-8.917525,1.621913,-3.575185,-0.872096,0.497231,14.139747,-32.294589,19.636111
138241,1079.973439,13.0,-7.107145,0.088564,0.157656,0.679648,0.016770,0.067401,48.881125,21.201545,16.077149,-8.528232,0.408745,0.935446,1.350346,0.818440,0.690475,53.932243,-6.843077,13.421438,-13.893424,-3.837934,-2.015418,-0.781569,0.577634,8.415215,23.761301,33.966283,-5.172884,-18.785659,-12.105381,1.143876,-3.564916,-1.137270,0.500805,13.950178,-32.510571,19.289818
138242,1079.981251,13.0,-5.492186,0.812349,-0.756811,0.773560,-0.017682,0.222706,48.973870,21.204928,16.348996,-7.211954,3.103333,0.872675,1.271846,0.811377,0.456124,53.760746,-7.150689,13.673213,-13.829675,-3.285212,-1.372541,-0.719143,0.665479,8.589082,25.878291,32.229446,-4.689766,-16.641929,-14.835902,0.112

### Simplify dataframe

In [32]:
# trim length of trial (number of rows in df)
obs_samples_trial = trial_length*60*frequency
df_2 = df_2.tail(obs_samples_trial) # keep last n samples to exclude speed transitions

# downsample trial
obs_samples_freq = int(128/frequency)

df_2 = df_2.iloc[::obs_samples_freq,:]
df_2

,Time(s)__,ActivityCode__,Waist_Acceleration_x,Waist_Acceleration_y,Waist_Acceleration_z,Waist_AngularVelocity_x,Waist_AngularVelocity_y,Waist_AngularVelocity_z,Waist_MagneticField_x,Waist_MagneticField_y,Waist_MagneticField_z,Chest_Acceleration_x,Chest_Acceleration_y,Chest_Acceleration_z,Chest_AngularVelocity_x,Chest_AngularVelocity_y,Chest_AngularVelocity_z,Chest_MagneticField_x,Chest_MagneticField_y,Chest_MagneticField_z,LeftAnkle_Acceleration_x,LeftAnkle_Acceleration_y,LeftAnkle_Acceleration_z,LeftAnkle_AngularVelocity_x,LeftAnkle_AngularVelocity_y,LeftAnkle_AngularVelocity_z,LeftAnkle_MagneticField_x,LeftAnkle_MagneticField_y,LeftAnkle_MagneticField_z,RightAnkle_Acceleration_x,RightAnkle_Acceleration_y,RightAnkle_Acceleration_z,RightAnkle_AngularVelocity_x,RightAnkle_AngularVelocity_y,RightAnkle_AngularVelocity_z,RightAnkle_MagneticField_x,RightAnkle_MagneticField_y,RightAnkle_MagneticField_z
99845,780.016406,13.0,-16.943086,-0.866212,1.550639,1.695432,-0.041715,0.142128,46.824701,17.829872,19.781106,-16.511537,-3.779138,2.748994,-0.617745,1.002580,1.231777,56.314810,-3.657905,9.205205,-2.616930,1.306196,-1.721131,-0.953777,0.060913,5.533746,9.798503,37.850352,-7.720855,-19.351680,2.802056,-8.656910,-1.827784,0.037838,0.809307,17.376410,-27.542949,23.993630
99846,780.024218,13.0,-17.014364,-1.073619,2.034535,1.572138,-0.039083,0.136383,46.802485,17.685903,19.797380,-17.894211,-4.114596,2.414327,-0.356821,1.013559,1.388567,56.166005,-3.715598,9.139307,-3.535728,1.701559,-1.807610,-0.937767,0.105216,5.965606,11.450614,37.719172,-7.442810,-20.404016,-0.340919,-6.105413,-2.036649,-0.062430,0.786526,16.970419,-27.992150,23.821884
99847,780.032030,13.0,-16.125338,-1.455629,2.049192,1.406735,-0.030359,0.127276,46.889820,17.833872,19.810591,-18.085142,-3.746080,2.157518,-0.045505,0.982700,1.414923,55.897150,-4.434903,9.514294,-4.679676,1.702107,-2.226846,-0.915898,0.170855,6.381192,13.208016,37.406598,-6.915831,-20.168940,-0.938388,-4.718499,-2.287510,-0.161653,0.670517,16.469931,-28.543070,23.389218
99848,780.039842,13.0,-13.262236,-1.411381,1.325144,1.247606,0.123353,0.151480,47.101039,17.810285,19.716666,-17.235366,-2.995088,1.950860,0.311389,0.944185,1.302557,55.596343,-4.744060,9.699715,-6.800508,1.213784,-2.116675,-0.729611,0.185139,6.754324,14.778611,36.742597,-6.571002,-20.326391,-0.913397,-2.514378,-2.420955,-0.240894,0.630739,16.328967,-28.979795,23.000532
99849,780.047654,13.0,-12.107939,-0.616791,0.923273,1.039678,0.044897,0.170054,47.186566,17.779456,19.509029,-15.445642,-2.445728,1.804393,0.641467,0.902080,1.091741,55.572371,-4.771042,9.740509,-8.695882,0.992947,-1.632674,-0.489227,0.122581,7.097539,16.549923,36.212878,-6.190411,-20.402232,-1.815646,-0.425656,-2.551367,-0.337738,0.600725,16.292575,-29.282384,22.658040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138240,1079.965626,13.0,-9.172976,-0.071055,0.946888,0.886438,-0.051979,-0.049732,48.758513,21.075537,16.244491,-10.439755,-2.226243,0.697521,1.214866,0.828242,0.970492,54.600461,-6.479572,13.343407,-13.206731,-2.503830,-3.527720,-1.010973,0.489969,8.172326,21.508113,35.098387,-5.653059,-20.794520,-8.917525,1.621913,-3.575185,-0.872096,0.497231,14.139747,-32.294589,19.636111
138241,1079.973439,13.0,-7.107145,0.088564,0.157656,0.679648,0.016770,0.067401,48.881125,21.201545,16.077149,-8.528232,0.408745,0.935446,1.350346,0.818440,0.690475,53.932243,-6.843077,13.421438,-13.893424,-3.837934,-2.015418,-0.781569,0.577634,8.415215,23.761301,33.966283,-5.172884,-18.785659,-12.105381,1.143876,-3.564916,-1.137270,0.500805,13.950178,-32.510571,19.289818
138242,1079.981251,13.0,-5.492186,0.812349,-0.756811,0.773560,-0.017682,0.222706,48.973870,21.204928,16.348996,-7.211954,3.103333,0.872675,1.271846,0.811377,0.456124,53.760746,-7.150689,13.673213,-13.829675,-3.285212,-1.372541,-0.719143,0.665479,8.589082,25.878291,32.229446,-4.689766,-16.641929,-14.835902,0.1128

Only include sensor data for model training and testing; remove time and activity code columns

In [33]:
df_2_data = df_2.iloc[:,2:] 
df_2_data

,Waist_Acceleration_x,Waist_Acceleration_y,Waist_Acceleration_z,Waist_AngularVelocity_x,Waist_AngularVelocity_y,Waist_AngularVelocity_z,Waist_MagneticField_x,Waist_MagneticField_y,Waist_MagneticField_z,Chest_Acceleration_x,Chest_Acceleration_y,Chest_Acceleration_z,Chest_AngularVelocity_x,Chest_AngularVelocity_y,Chest_AngularVelocity_z,Chest_MagneticField_x,Chest_MagneticField_y,Chest_MagneticField_z,LeftAnkle_Acceleration_x,LeftAnkle_Acceleration_y,LeftAnkle_Acceleration_z,LeftAnkle_AngularVelocity_x,LeftAnkle_AngularVelocity_y,LeftAnkle_AngularVelocity_z,LeftAnkle_MagneticField_x,LeftAnkle_MagneticField_y,LeftAnkle_MagneticField_z,RightAnkle_Acceleration_x,RightAnkle_Acceleration_y,RightAnkle_Acceleration_z,RightAnkle_AngularVelocity_x,RightAnkle_AngularVelocity_y,RightAnkle_AngularVelocity_z,RightAnkle_MagneticField_x,RightAnkle_MagneticField_y,RightAnkle_MagneticField_z
99845,-16.943086,-0.866212,1.550639,1.695432,-0.041715,0.142128,46.824701,17.829872,19.781106,-16.511537,-3.779138,2.748994,-0.617745,1.002580,1.231777,56.314810,-3.657905,9.205205,-2.616930,1.306196,-1.721131,-0.953777,0.060913,5.533746,9.798503,37.850352,-7.720855,-19.351680,2.802056,-8.656910,-1.827784,0.037838,0.809307,17.376410,-27.542949,23.993630
99846,-17.014364,-1.073619,2.034535,1.572138,-0.039083,0.136383,46.802485,17.685903,19.797380,-17.894211,-4.114596,2.414327,-0.356821,1.013559,1.388567,56.166005,-3.715598,9.139307,-3.535728,1.701559,-1.807610,-0.937767,0.105216,5.965606,11.450614,37.719172,-7.442810,-20.404016,-0.340919,-6.105413,-2.036649,-0.062430,0.786526,16.970419,-27.992150,23.821884
99847,-16.125338,-1.455629,2.049192,1.406735,-0.030359,0.127276,46.889820,17.833872,19.810591,-18.085142,-3.746080,2.157518,-0.045505,0.982700,1.414923,55.897150,-4.434903,9.514294,-4.679676,1.702107,-2.226846,-0.915898,0.170855,6.381192,13.208016,37.406598,-6.915831,-20.168940,-0.938388,-4.718499,-2.287510,-0.161653,0.670517,16.469931,-28.543070,23.389218
99848,-13.262236,-1.411381,1.325144,1.247606,0.123353,0.151480,47.101039,17.810285,19.716666,-17.235366,-2.995088,1.950860,0.311389,0.944185,1.302557,55.596343,-4.744060,9.699715,-6.800508,1.213784,-2.116675,-0.729611,0.185139,6.754324,14.778611,36.742597,-6.571002,-20.326391,-0.913397,-2.514378,-2.420955,-0.240894,0.630739,16.328967,-28.979795,23.000532
99849,-12.107939,-0.616791,0.923273,1.039678,0.044897,0.170054,47.186566,17.779456,19.509029,-15.445642,-2.445728,1.804393,0.641467,0.902080,1.091741,55.572371,-4.771042,9.740509,-8.695882,0.992947,-1.632674,-0.489227,0.122581,7.097539,16.549923,36.212878,-6.190411,-20.402232,-1.815646,-0.425656,-2.551367,-0.337738,0.600725,16.292575,-29.282384,22.658040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138240,-9.172976,-0.071055,0.946888,0.886438,-0.051979,-0.049732,48.758513,21.075537,16.244491,-10.439755,-2.226243,0.697521,1.214866,0.828242,0.970492,54.600461,-6.479572,13.343407,-13.206731,-2.503830,-3.527720,-1.010973,0.489969,8.172326,21.508113,35.098387,-5.653059,-20.794520,-8.917525,1.621913,-3.575185,-0.872096,0.497231,14.139747,-32.294589,19.636111
138241,-7.107145,0.088564,0.157656,0.679648,0.016770,0.067401,48.881125,21.201545,16.077149,-8.528232,0.408745,0.935446,1.350346,0.818440,0.690475,53.932243,-6.843077,13.421438,-13.893424,-3.837934,-2.015418,-0.781569,0.577634,8.415215,23.761301,33.966283,-5.172884,-18.785659,-12.105381,1.143876,-3.564916,-1.137270,0.500805,13.950178,-32.510571,19.289818
138242,-5.492186,0.812349,-0.756811,0.773560,-0.017682,0.222706,48.973870,21.204928,16.348996,-7.211954,3.103333,0.872675,1.271846,0.811377,0.456124,53.760746,-7.150689,13.673213,-13.829675,-3.285212,-1.372541,-0.719143,0.665479,8.589082,25.878291,32.229446,-4.689766,-16.641929,-14.835902,0.112863,-3.377444,-1.316918,0.488785,14.000116,-32.912712,18.152025
138243,-5.119762,1.332467,-0.284706,0.913211,-0.240359,0.331789,49.048208,21.092482,16.463311,-7.5362

In [35]:
# convert pandas dataframe to numpy array
load_X = df_2_data.to_numpy()
load_X.shape

(38400, 36)

## Set up sensors

In [36]:
from random import choice

lags = frequency # length of trajectory used to train LSTM; chose 128 for Ingraham data sampled at 128 Hz
n = load_X.shape[0] # total number of time steps (observations)
m = load_X.shape[1] # number of features per time step

time = np.arange(1, n+1, 1)

## Visualize IMU data

Observing raw data is important for understanding what is being used to train and test models. We visualize data using altair (alt). Two tutorials on some basic functionality are linked below:

* Long tutorial (1hr): https://youtu.be/umTwkgQoo_E

* Short tutorial (20min): https://youtu.be/o-nVM_FdIVc

Uncomment the line below when code is fully functioning to disable the 5000-row dataframe limit

In [ ]:
# alt.data_transformers.disable_max_rows()

In [37]:
# set time to start at 0 (optional for clean viz)
time_zeroed = df_2.loc[:,"Time(s)__"] - df_2["Time(s)__"].iloc[0]

# view the first portion of the trial
df_2_data_reduced = df_2.head(1000)
df_2_data_reduced['Time_Zeroed(s)'] = time_zeroed.head(1000)

# view the last portion of the trial
#df_2_data_reduced = df_2.tail(4000) 
#df_2_data_reduced['Time_Zeroed(s)'] = time_zeroed.tail(4000)

df_2_data_reduced.head()

/var/folders/qr/twbsyh_d3dg3vdbgf28pxhfm0000gn/T/ipykernel_70643/1697600741.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_data_reduced['Time_Zeroed(s)'] = time_zeroed.head(1000)


,Time(s)__,ActivityCode__,Waist_Acceleration_x,Waist_Acceleration_y,Waist_Acceleration_z,Waist_AngularVelocity_x,Waist_AngularVelocity_y,Waist_AngularVelocity_z,Waist_MagneticField_x,Waist_MagneticField_y,Waist_MagneticField_z,Chest_Acceleration_x,Chest_Acceleration_y,Chest_Acceleration_z,Chest_AngularVelocity_x,Chest_AngularVelocity_y,Chest_AngularVelocity_z,Chest_MagneticField_x,Chest_MagneticField_y,Chest_MagneticField_z,LeftAnkle_Acceleration_x,LeftAnkle_Acceleration_y,LeftAnkle_Acceleration_z,LeftAnkle_AngularVelocity_x,LeftAnkle_AngularVelocity_y,LeftAnkle_AngularVelocity_z,LeftAnkle_MagneticField_x,LeftAnkle_MagneticField_y,LeftAnkle_MagneticField_z,RightAnkle_Acceleration_x,RightAnkle_Acceleration_y,RightAnkle_Acceleration_z,RightAnkle_AngularVelocity_x,RightAnkle_AngularVelocity_y,RightAnkle_AngularVelocity_z,RightAnkle_MagneticField_x,RightAnkle_MagneticField_y,RightAnkle_MagneticField_z,Time_Zeroed(s)
99845,780.016406,13.0,-16.943086,-0.866212,1.550639,1.695432,-0.041715,0.142128,46.824701,17.829872,19.781106,-16.511537,-3.779138,2.748994,-0.617745,1.002580,1.231777,56.314810,-3.657905,9.205205,-2.616930,1.306196,-1.721131,-0.953777,0.060913,5.533746,9.798503,37.850352,-7.720855,-19.351680,2.802056,-8.656910,-1.827784,0.037838,0.809307,17.376410,-27.542949,23.993630,0.000000
99846,780.024218,13.0,-17.014364,-1.073619,2.034535,1.572138,-0.039083,0.136383,46.802485,17.685903,19.797380,-17.894211,-4.114596,2.414327,-0.356821,1.013559,1.388567,56.166005,-3.715598,9.139307,-3.535728,1.701559,-1.807610,-0.937767,0.105216,5.965606,11.450614,37.719172,-7.442810,-20.404016,-0.340919,-6.105413,-2.036649,-0.062430,0.786526,16.970419,-27.992150,23.821884,0.007812
99847,780.032030,13.0,-16.125338,-1.455629,2.049192,1.406735,-0.030359,0.127276,46.889820,17.833872,19.810591,-18.085142,-3.746080,2.157518,-0.045505,0.982700,1.414923,55.897150,-4.434903,9.514294,-4.679676,1.702107,-2.226846,-0.915898,0.170855,6.381192,13.208016,37.406598,-6.915831,-20.168940,-0.938388,-4.718499,-2.287510,-0.161653,0.670517,16.469931,-28.543070,23.389218,0.015624
99848,780.039842,13.0,-13.262236,-1.411381,1.325144,1.247606,0.123353,0.151480,47.101039,17.810285,19.716666,-17.235366,-2.995088,1.950860,0.311389,0.944185,1.302557,55.596343,-4.744060,9.699715,-6.800508,1.213784,-2.116675,-0.729611,0.185139,6.754324,14.778611,36.742597,-6.571002,-20.326391,-0.913397,-2.514378,-2.420955,-0.240894,0.630739,16.328967,-28.979795,23.000532,0.023437
99849,780.047654,13.0,-12.107939,-0.616791,0.923273,1.039678,0.044897,0.170054,47.186566,17.779456,19.509029,-15.445642,-2.445728,1.804393,0.641467,0.902080,1.091741,55.572371,-4.771042,9.740509,-8.695882,0.992947,-1.632674,-0.489227,0.122581,7.097539,16.549923,36.212878,-6.190411,-20.402232,-1.815646,-0.425656,-2.551367,-0.337738,0.600725,16.292575,-29.282384,22.658040,0.031249


Select which sensor location to visualize.

In [ ]:
location = 'RightAnkle' # RightAnkle, LeftAnkle, Chest, Waist

In [47]:
# Define signal types and axes.
sensor = ['Acceleration', 'AngularVelocity', 'MagneticField']
dir = ['x','y','z']
plotStack = [0,0,0] # Preallocate plot for each signal

# Generate plots for each sensor type
for iSensor in range(len(sensor)): # loop through the signal types
    # create plots for x,y,z directions
    x_signal = alt.Chart(df_2_data_reduced).mark_line().encode(
        x = 'Time_Zeroed(s)',
        y = alt.Y(location + '_' + sensor[iSensor] + '_x', title = sensor[iSensor]),
        color = alt.value('#c6dbef')
    ).properties(
        width = 1000,
        height = 200
    )
    y_signal = alt.Chart(df_2_data_reduced).mark_line().encode(
        x = 'Time_Zeroed(s)',
        y = alt.Y(location + '_' + sensor[iSensor] + '_y', title = sensor[iSensor]),
        color = alt.value("#6baed6")
    )
    z_signal = alt.Chart(df_2_data_reduced).mark_line().encode(
        x = 'Time_Zeroed(s)',
        y = alt.Y(location + '_' + sensor[iSensor] + '_z', title = sensor[iSensor]),
        color = alt.value("#08519c")
    ).interactive()
    # Combine x,y,z plots
    plotStack[iSensor] = x_signal + y_signal + z_signal

alt.vconcat(plotStack[0], plotStack[1], plotStack[2]).properties(title = [location,""])

alt.VConcatChart(...)

# SHRED model function

In [87]:
### Generate input sequences to a SHRED model
def train_SHRED_model(transformed_X, sc, train_indices, valid_indices, test_indices, sensor_locations, num_sensors, m, n, lags):
  """
    Trains SHRED model for time series reconstruction

  Args: 
    transformed_X (numpy array): MinMax scaled dataset.
    sc (MinMaxScaler): Fitted MinMax scaler for inverse transformation.
    train_indices (array): Indices for the training set.
    valid_indices (array): Indices for the validation set.
    test_indices (array): Indices for the test set.
    sensor_locations (array): Column indices for sensor data.
    num_sensors (int): Number of signal measurements from sensors (e.g, triaxial = 3)
    m (int): Number of features per timestep
    n (int): Total number of time steps (observations)
    lags (int): length of trajectory
    
  Return:
    test_recons: Reconstructed data from the SHRED model on the test set
    test_ground_truth: Ground truth data from the test set
  """

  all_data_in = np.zeros((n - lags, lags, num_sensors))
  for i in range(len(all_data_in)):
      all_data_in[i] = transformed_X[i:i+lags, sensor_locations]
  ### Generate training validation and test datasets both for reconstruction of states and forecasting sensors
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  train_data_in = torch.tensor(all_data_in[train_indices], dtype=torch.float32).to(device)
  valid_data_in = torch.tensor(all_data_in[valid_indices], dtype=torch.float32).to(device)
  test_data_in = torch.tensor(all_data_in[test_indices], dtype=torch.float32).to(device)

  ### -1 to have output be at the same time as final sensor measurements
  train_data_out = torch.tensor(transformed_X[train_indices + lags - 1], dtype=torch.float32).to(device)
  valid_data_out = torch.tensor(transformed_X[valid_indices + lags - 1], dtype=torch.float32).to(device)
  test_data_out = torch.tensor(transformed_X[test_indices + lags - 1], dtype=torch.float32).to(device)

  train_dataset = TimeSeriesDataset(train_data_in, train_data_out)
  valid_dataset = TimeSeriesDataset(valid_data_in, valid_data_out)
  test_dataset = TimeSeriesDataset(test_data_in, test_data_out)
  shred = models.SHRED(num_sensors, m, hidden_size=64, hidden_layers=2, l1=350, l2=400, dropout=0.1).to(device)
  validation_errors = models.fit(shred, train_dataset, valid_dataset, batch_size=64, num_epochs=500, lr=1e-3, verbose=True, patience=5)

  # Generate reconstructions from the test set and print mean square error compared to the ground truth
  test_recons = sc.inverse_transform(shred(test_dataset.X).detach().cpu().numpy())
  test_ground_truth = sc.inverse_transform(test_dataset.Y.detach().cpu().numpy())

  return test_recons, test_ground_truth

## Results

#### Compute Error

In [53]:
def rmse_error(Ypred, Ytest):
    """
        Computes the Root Mean Squared Error between two data arrays
        
    Args:
        Ypred (array): Predicted values
        Ytest (array): True values
    
    Return:
        err: RSME value
    """
    
    err = np.sqrt(((Ypred - Ytest) ** 2).mean()) 

    return err

def mae_error(Ypred, Ytest):
    """
        Computes the Mean Absolute Error between two data arrays
        
    Args:
        Ypred (array): Predicted values
        Ytest (array): True values
    
    Return:
        err: MAE value
    """
    
    err = abs((Ypred - Ytest).mean()) 

    return err

def mbe_error(Ypred, Ytest):
    """
        Computes the Mean Bias Error between two data arrays
        
    Args:
        Ypred (array): Predicted values
        Ytest (array): True values
    
    Return:
        err: MBE value
    """
    
    err = (Ypred - Ytest).mean()

    return err

#### Visualize Waveform

In [54]:
def concatRaw(sensor_place, sensor_path, start, end, tmp_Ypred, tmp_Ytest, subj):

    """
        Concatenates raw prediction and test data into a DataFrame with detailed sensor and output information.

    Args:
        sensor_place (str): Location of the sensor (e.g., "Waist", "Chest").
        sensor_path (str): Path indicating the type of sensor data (e.g., '3acc_Training').
        start (int): Start index for processing subjects.
        end (int): End index for processing subjects.
        tmp_Ypred (DataFrame): Predicted output values.
        tmp_Ytest (DataFrame): True output values for validation.
        subj (str): Subject identifier.

    Returns:
        df_mse_new: A DataFrame containing detailed sensor and error information.
    """

    df_mse = pd.DataFrame()
    columns = ['Subject', 'Pred', 'True', 'Input Location', 'Sensor Type', 'Output Location', 'Output Signal', 'Output Direction', 'Output Axis', 'Assessment']
    df_mse_new = pd.DataFrame(columns=columns)
    num_states = 36 # number of features hardcoded, consider adding input to function def line

    for i in range(start, end+1):
        # if i < 10:
        #     subj = 'P0'+str(i)
        # elif i >= 10:
        #     subj = 'P'+str(i)


        tmp = np.zeros([1,num_states])
        for i in range(0,num_states):
            tmp[:,i] = rmse_error(tmp_Ytest.iloc[:,i],tmp_Ypred.iloc[:,i])

        time_vec = tmp_Ytest['Time']

        tmp_mse = pd.DataFrame(tmp, columns = tmp_Ypred.columns[:num_states])
        #########################################################
        # create df with one error observation per row
        # add columns for subject, value, input location, input signal, output location, output signal, output direction, output axis, Assessment
        
        if sensor_path == '3acc_Training':
            sensor_type = 'Triaxial Accelerometer'
        elif sensor_path == '3acc3gyro_Training':
            sensor_type = 'Triaxial Accelerometer & Gyroscope'
        elif sensor_path == 'Xacc_Training':
            sensor_type = 'Uniaxial Accelerometer'
        else:
            sensor_type = 'Triaxial Gyroscope'

        row_count = 0
        time_ind = -1
        for row_Ypred in range(min(len(tmp_Ypred), 200)):
            for idx, value in tmp_Ypred.iloc[row_Ypred, :36].items():
                tmp_tidy = pd.DataFrame(columns = df_mse_new.columns)
                # assign subject
                tmp_tidy.loc[row_count,'Subject'] = subj
                # assign predicted value
                tmp_tidy.loc[row_count,'Pred'] = value
                # assign true value
                tmp_tidy.loc[row_count, 'True'] = tmp_Ytest[idx][row_Ypred]
                # assign input location
                tmp_tidy.loc[row_count,'Input Location'] = sensor_place
                # assign input signal
                tmp_tidy.loc[row_count,'Sensor Type'] = sensor_type
                # assign output location
                if 'Waist' in idx:
                    tmp_tidy.loc[row_count,'Output Location'] = 'Waist'
                elif 'Chest' in idx:
                    tmp_tidy.loc[row_count,'Output Location'] = 'Chest'
                elif 'LeftAnkle' in idx:
                    tmp_tidy.loc[row_count, 'Output Location'] = 'LeftAnkle'
                elif 'RightAnkle' in idx:
                    tmp_tidy.loc[row_count, 'Output Location'] = 'RightAnkle'

                # assign output signal
                if 'Accel' in idx:
                    tmp_tidy.loc[row_count, 'Output Signal'] = 'Acceleration'
                elif 'Angular' in idx:
                    tmp_tidy.loc[row_count, 'Output Signal'] = 'Angular Velocity'
                elif 'Magnetic' in idx:
                    tmp_tidy.loc[row_count, 'Output Signal'] = 'Magnetic Field'

                # assign output direction
                if 'Chest' in idx and '_y' in idx:
                    tmp_tidy.loc[row_count, 'Output Direction'] = 'ML'
                elif 'Chest' in idx and '_z' in idx:
                    tmp_tidy.loc[row_count, 'Output Direction'] = 'AP'
                elif '_y' in idx: # Waist and Ankles
                    tmp_tidy.loc[row_count, 'Output Direction'] = 'AP'
                elif '_z' in idx: # Waist and Ankles
                    tmp_tidy.loc[row_count, 'Output Direction'] = 'ML'
                else: # all others contain '_x'
                    tmp_tidy.loc[row_count, 'Output Direction'] = 'Vertical'

                # assign output axis
                if '_x' in idx:
                    tmp_tidy.loc[row_count, 'Output Axis'] = 'x'
                elif '_y' in idx:
                    tmp_tidy.loc[row_count, 'Output Axis'] = 'y'
                elif '_z' in idx:
                    tmp_tidy.loc[row_count, 'Output Axis'] = 'z'
                
                # assign assessment
                tmp_tidy.loc[row_count, 'Assessment'] = 'Single Speed'

                # assign time stamp
                if row_count % num_states == 0:
                    time_ind = time_ind + 1
                tmp_tidy.loc[row_count, 'Time'] = time_vec[time_ind]

                # concatenate with the previous iterations
                df_mse_new = pd.concat([df_mse_new, tmp_tidy])
                row_count = row_count + 1


    return df_mse_new

In [55]:
def extractSignal(signal_loc, signal_type, signal_ax, df_SHRED):

    """
        Combines the predicted and true sensor signal data for a given location, type, and axis.

    Args:
        signal_loc (str): The location of the sensor (e.g., 'Waist', 'Chest').
        signal_type (str): The type of signal (e.g., 'Acceleration', 'Angular Velocity').
        signal_ax (str): The axis of the sensor signal (e.g., 'x', 'y', 'z').
        df_SHRED (DataFrame): DataFrame containing SHRED model predictions and true values.

    Returns:
        df_signal: A DataFrame containing the time, predicted values (SHRED), and true values for the specified signal.
    """

    df_SHRED_signal = pd.DataFrame(columns = ['Type' , 'Time', 'Value'])
    df_true_signal = pd.DataFrame(columns = df_SHRED_signal.columns)

    # get SHRED data
    df_SHRED_reduced = df_SHRED[(df_SHRED['Output Axis'] == signal_ax) & (df_SHRED['Output Location'] == signal_loc) & (df_SHRED['Output Signal'] == signal_type)]
    df_SHRED_signal['Time'] = df_SHRED_reduced['Time']
    df_SHRED_signal['Value'] = df_SHRED_reduced['Pred']
    df_SHRED_signal['Type'] = 'SHRED'
    max_length = len(df_SHRED_signal)

    # get true data
    df_true_signal['Time'] = df_SHRED_reduced['Time']
    df_true_signal['Value'] = df_SHRED_reduced['True']
    df_true_signal['Type'] = 'True'

    # concatenate SHRED with true data
    df_signal = pd.concat([df_true_signal, df_SHRED_signal])

    return df_signal

# Train models

In [56]:
# partition into training, validation, test sets
train_indices, valid_indices, test_indices = ft.partition_data_seq(load_X, n, lags) 

# normalize input data using MinMaxScaler
transformed_X, sc = ft.transform_data(load_X, train_indices) 

### Define input sensor

In [57]:
# choose input sensor location
sensor_place = 'RightAnkle' # RightAnkle, Waist, or Chest

# choose input sensor type
sensor_path = '3acc_Training' # 3acc_Training, 3gyro_Training, 3acc3gyro_Training, or Xacc_Training

# access columns indices from main dataframe
sensor_locations, num_sensors = ft.sensor_loc_fun(sensor_path, sensor_place)
train_names = [df_2_data.columns[i] for i in sensor_locations]

print('Number of signals: ', num_sensors)
print('Signals were chosen at: ', sensor_place)
print('Signals chosen: ', [df_2_data.columns[i] for i in sensor_locations])

Number of signals:  3
Signals were chosen at:  RightAnkle
Signals chosen:  ['RightAnkle_Acceleration_x', 'RightAnkle_Acceleration_y', 'RightAnkle_Acceleration_z']


In [80]:
# check path for saving dataframes
if trial_length == 6 and frequency == 128: # full-length trial, full frequency
  save_train_df = dataframe_path+'/'+trial_path+'/'+AC_path+'/'+sensor_place+'/'+sensor_path+'/P'+subj+'_Ytest_SHRED_Train_'+sensor_place+'_'+sensor_path+'.csv'
  save_test_df = dataframe_path+'/'+trial_path+'/'+AC_path+'/'+sensor_place+'/'+sensor_path+'/P'+subj+'_Ypred_SHRED_Train_'+sensor_place+'_'+sensor_path+'.csv'
else: # reduced trial length or frequency
  save_train_df = dataframe_path+'/'+trial_path+'/'+AC_path+'/'+sensor_place+'/'+sensor_path+'/P'+subj+'_Ypred_SHRED_Train_'+sensor_place+'_'+sensor_path+'_'+save_tag+'.csv'
  save_test_df = dataframe_path+'/'+trial_path+'/'+AC_path+'/'+sensor_place+'/'+sensor_path+'/P'+subj+'_Ypred_SHRED_Train_'+sensor_place+'_'+sensor_path+'_'+save_tag+'.csv'

print(os.path.isdir(save_test_df))
print(save_train_df)
print(save_test_df)

False
/Users/davidgreen/SHRED/Datasets/Dataframes/Ingraham_IMU_reconstruction_nowrist_running/AC13/RightAnkle/3acc_Training/P01_Ypred_SHRED_Train_RightAnkle_3acc_Training_5min.csv
/Users/davidgreen/SHRED/Datasets/Dataframes/Ingraham_IMU_reconstruction_nowrist_running/AC13/RightAnkle/3acc_Training/P01_Ypred_SHRED_Train_RightAnkle_3acc_Training_5min.csv


In [81]:
# train SHRED model
Ypred, Ytest = train_SHRED_model(transformed_X, sc, train_indices, valid_indices, test_indices, sensor_locations, num_sensors, m, n, lags)

df_Ytest_SHRED = pd.DataFrame(Ytest, columns = df_2_data.columns)
df_Ypred_SHRED = pd.DataFrame(Ypred, columns = df_2_data.columns)

df_Ytest_SHRED['Type']='Measured'
df_Ypred_SHRED['Type']='SHRED'

df_Ytest_SHRED['Time']=df_2.iloc[test_indices + lags - 1,0].to_numpy()
df_Ypred_SHRED['Time']=df_2.iloc[test_indices + lags - 1,0].to_numpy()

# save dataframes as .csv if specified
if save_df == True:
  df_Ytest_SHRED.to_csv(save_train_df)
  df_Ypred_SHRED.to_csv(save_test_df)


Training epoch 1
Error tensor(0.2296)
Training epoch 20
Error tensor(0.0875)
Training epoch 40
Error tensor(0.0827)
Training epoch 60
Error tensor(0.0795)
Training epoch 80
Error tensor(0.0802)
Training epoch 100
Error tensor(0.0788)
Training epoch 120
Error tensor(0.0799)
Training epoch 140
Error tensor(0.0811)
Training epoch 160
Error tensor(0.0816)
Training epoch 180
Error tensor(0.0810)
Training epoch 200
Error tensor(0.0786)
Training epoch 220
Error tensor(0.0801)
Training epoch 240
Error tensor(0.0806)
Training epoch 260
Error tensor(0.0798)
Training epoch 280
Error tensor(0.0803)
Training epoch 300
Error tensor(0.0809)


# Visualize Results

In [82]:
df_SHRED_tidy = ft.concatRaw(sensor_place, sensor_path, 1, 1, df_Ypred_SHRED, df_Ytest_SHRED, subj)

In [90]:
# format: extractSignal(output_location, output_signal, output_axis, df_SHRED_tidy)
    # output_location: 'Chest', 'Waist', 'RightAnkle', 'LeftAnkle'
    # output_signal: 'Acceleration', 'AngularVelocity', 'Magnetic Field'
    # output_axis: 'x', 'y', z'

Signal1 = ft.extractSignal('LeftAnkle', 'Acceleration', 'x', df_SHRED_tidy)
Signal2 = ft.extractSignal('Chest', 'Acceleration', 'x', df_SHRED_tidy)

my_scheme = ['#1e88e5', '#878787'] # '#014337', '#1e88e5', '#DB1048'

Signal1_rmse = ft.rmse_error(Signal1[Signal1['Type'] == 'True']['Value'], Signal1[Signal1['Type'] == 'SHRED']['Value'])
Signal2_rmse = ft.rmse_error(Signal2[Signal2['Type'] == 'True']['Value'], Signal2[Signal2['Type'] == 'SHRED']['Value'])

# plot left ankle acceleration
line1 = alt.Chart(Signal1).mark_line().encode(
    x=alt.X('Time:Q', title='Time (s)'),
    y=alt.Y('Value:Q', title='Acceleration (m/s\u00b2)'),
    color=alt.Color('Type:N', scale=alt.Scale(range=my_scheme))
).properties(
    width=600,
    height=400,
    title=f'Signal 1: RMSE = {Signal1_rmse:.2f}'  # Can change this title to be specific to the output signal
)
# plot chest acceleration
line2 = alt.Chart(Signal2).mark_line().encode(
    x=alt.X('Time:Q', title='Time (s)'),
    y=alt.Y('Value:Q', title='Acceleration (m/s\u00b2)'),
    color=alt.Color('Type:N', scale=alt.Scale(range=my_scheme))
).properties(
    width=600,
    height=400,
    title=f'Signal 2: RMSE = {Signal2_rmse:.2f}'  # Can change this title to be specific to the output signal
)

final_chart = alt.vconcat(line2, line1).properties(
    title=f'Parameter = {save_tag}, Right Ankle Input' # Can change this title to match the input sensor
    # increase font size
).configure_axis(
    labelFontSize=18,
    titleFontSize=20
).configure_title(
    fontSize=24
).configure_legend(
    labelFontSize=18,
    titleFontSize=20
)

final_chart

alt.VConcatChart(...)